# Intro to SQL

Sourced largely from the CSSIP-AIR Big Data course's [Data and Databases](https://github.com/CSSIP-AIR/Big-Data-Workbooks/blob/master/02.%20Database%20Basics/Data_and_databases.ipynb) notebook.

# Table of Contents

- [Introduction](#Introduction)

    - [Learning objectives](#Learning-objectives)
    - [Tables we will look at](#Tables-we-will-look-at)

        - [Administrative data](#Administrative-data)

            - [Illinois Department of Corrections exit dataset](#ildoc_exit)
            - [IL DOC admissions](#ildoc_admit)
            - [IL DOC parolee last known address](#il_doc_parole)
            - [IL Department of Employment Security wage records](#il_wage)

        - [Public data](#Public-data)

            - [Zip code tabulation areas](#Zip-code-tabulation-areas)
            - [Census Blocks](#Census-Block-polygons)

- [Setup](#Setup)
- [SQL basics](#SQL-basics)

    - [Querying the database](#Querying-the-database)
    - [**Exercise 1**](#Exercise-1)
    - [WHERE clauses: Limiting the results](#WHERE-clauses:-Limiting-the-results)
    - [**Exercise 2**](#Exercise-2)
    - [JOIN: Connecting multiple tables](#JOIN:-Connecting-multiple-tables)
    - [GROUP BY and aggregate functions](#GROUP-BY-and-aggregate-functions)
    - [**Exercise 3**](#Exercise-3)
    - [ORDER BY](#ORDER-BY)
    
- [PostGIS](#PostGIS)
    - [PostGIS examples](#PostGIS-examples)
    
        - [Example 1: Points in polygons](#Example-1:-Points-in-polygons)
        - [Example 2: Nearest neighbor distances](#Example-2:-Nearest-neighbor-distances)
    
- [Addendum - Other useful SQL](#Addendum---Other-useful-SQL)

    - [Listing tables and columns](#Listing-tables-and-columns)
    - [Modifying the database](#Modifying-the-database)
    - [Coordinate-Reference Systems](#Coordinate-Reference-Systems)
    - [Get spatial information from PostGIS](#Get-spatial-information-from-PostGIS)

# Introduction

- Back to the [Table of Contents](#Table-of-Contents)

In this notebook, we will introduce structured query language (SQL).  SQL is the main way one interacts with relational databases.  SQL is much different from traditional programming languages.  A single SQL statement can contain the complexity of an entire Python or Java program, and the syntax initially seems relatively straightforward, but can quickly become confusing as you try more complicated queries.

We will learn the basics of SQL, then use it to provide a pattern for exploring the class data sets, focusing on better understanding the Illinois Department of Corrections exit data and an open source database of zip code tabulation areas.

## Learning objectives

- Back to the [Table of Contents](#Table-of-Contents)

Learning objectives:

- Become familiar with the basic syntax, structure, and uses of SQL.
- Get some hands on experience writing and running SQL queries.
- Use and learn descriptive SQL queries that can help you familiarize yourself with a data set.

## Tables we will look at

- Back to the [Table of Contents](#Table-of-Contents)

In this notebook, we will use tables in the "`appliedda`" database. These tables are based on data provided by a few different agencies: IL Department of Corrections, IL Department of Employment Statistics, US Department of Housing and Urban Development, and the U.S. Census Bureau.

Specifically, we will be connecting to the "`appliedda`" database and looking at the "`ildoc_admit`", "`ildoc_exit`", and "`il_wage`" tablea, and the zip code table TK.  Basic information on these tables follows. 

### Administrative data

- Back to the [Table of Contents](#Table-of-Contents)

#### ildoc_admit

- Back to the [Table of Contents](#Table-of-Contents)

This dataset includes admission data from the IL Department of Corrections from 1990 - 2015, that is: individual records of all people who were admitted to Illinois correctional facilities during that 16 year period. The data include many other fields of information (~170 total) including demographic data; charges and sentencing information; and conduct, security level, and health information. The full data description is available outside the ADRF on the course website, and inside the ADRF in the [ADRF Explorer](https://deepdish.adrf.info/detail/adrf-000001).

#### ildoc_exit

- Back to the [Table of Contents](#Table-of-Contents)

This dataset includes exit data from the IL Department of Corrections from 1990 - 2015, that is: individual records of all people who were released from Illinois correctional facilities during that 16 year period. The data include many other fields of information (~170 total) including admission dates, original and revised admission types, expected release dates, participation in programs offered by the correctional facility, and more. The full data description is available outside the ADRF on the course website, and inside the ADRF in the [ADRF Explorer](https://deepdish.adrf.info/detail/adrf-000002).

#### il_doc_parole

- Back to the [Table of Contents](#Table-of-Contents)

Location information for persons active in the Illinois parole system from 2009 to 2015. Note there are anumber of cautions associated with this data which can be found on the [ADRF Explorer](https://deepdish.adrf.info/detail/adrf-000036) page for this dataset.


#### il_wage

- Back to the [Table of Contents](#Table-of-Contents)

This dataset includes quarterly records of wages for every job held by each person in the state of Illinois from 2005 to 2015.  This data is derived from the Illinois Department of Employment Security (IDES) Unemployment Insurance (UI) wage file that the Local Employment Dynamics (LED) state partners supply to the Census department for use in producing Quarterly Workforce Indicators (QWI).  The full data description is available outside the ADRF on the course website.

### Public data

- Back to the [Table of Contents](#Table-of-Contents)

#### Zip code tabulation areas

- Back to the [Table of Contents](#Table-of-Contents)

Zip code tabulation areas (ZCTAs) are the same 5-digit zip codes everyone is used to, but only those which have a geographic boundary assigned to them (some zip codes represent specific buildings or entities so are technically contained within the boundary of a different zip code). This data will be useful when considering geographic distributions of ex-offenders.

#### Census Block polygons

- Back to the [Table of Contents](#Table-of-Contents)

Census Blocks are the smallest unit provided by the US Census and is designated by a 15 digit unique code known as its FIPS (Federal Information Processing Standard) code. Most geographic areas maintained by the federal government are simply groupings of Blocks and are used at varying levels depending on what analysis is being performed at what geographic scale. Geographic boundary files are provided for each of the following (and more):
* `##` - state code (eg '17' for Illinois)
* `###` - county code within a given state (eg '031' for Cook County in IL)
* `###` - sub-county code within a given county
* `###` - Tract code within a given sub-county (so to uniquely identify a Tract you need 11 digits)
* `#` - Block Group code within a given Tract
* `###` - Block code within a given Block Group

# Setup

Before you begin, make sure to pick a method of running SQL queries against the class database from the [Database clients notebook](./data_and_databases-Database_clients.ipynb) and get connected to the `appliedda` database.  If you are new to SQL, we recommend working with pgAdmin to start - it is the easiest and most intuitive of the ways you can run SQL in the ADRF.

# SQL basics

- Back to the [Table of Contents](#Table-of-Contents)

SQL is a quirky language. It is different from procedural languages like Python and is designed for a very specific purpose: to interact with relational data. It isn't structured like other languages, and while it can make data access easy, it also can make tasks that would be easy in other languages (though perhaps not exceptionally performant) confoundingly complex.  Let's dive in so you can see it for yourself!

## Querying the database

- Back to the [Table of Contents](#Table-of-Contents)

The basic method of querying the database is to use a select statement:

    SELECT *
    FROM ildoc_exit
    LIMIT 1000; 

Where:

- Columns or variables that would like returned are put in the SELECT clause (after the word "SELECT" but before the word "FROM").  An asterisk ( "\*" ) is a wildcard - it will return all columns for a given table.
- The name of the table (or names of the tables - more on this in a bit) you want to query is put after the word "FROM", in the FROM clause.
- It is considered good style to capitalize words in an SQL query that are SQL words, not variables, table names, or values you are filtering on or searching for, ie. SELECT, FROM, WHERE, etc.
- Although it isn't always necessary in PostgreSQL, you should end SQL statements with a semi-colon.  It isn't required everywhere, but it is required in some contexts so better to be aware and get into the habit.
- "`LIMIT 1000`" is included throughout so we don't try to retrieve an entire table, in case our database client of choice loads results to memory.

Instead of specifying “all” columns ( "\*" ), you can specify which columns you want by name, in a comma-delimited list after "SELECT" (here adding admission year - curadmyr - and discharge year - actdisyr):

    SELECT docnbr, curadmyr, actdisyr
    FROM ildoc_exit
    LIMIT 1000;

You can specify calculations in the list of columns also (time in jail = discharge year - admission year):

    SELECT docnbr, curadmyr, actdisyr, ( actdisyr - curadmyr )
    FROM ildoc_exit
    LIMIT 1000;

And you can give those new columns names:

    SELECT docnbr, curadmyr, actdisyr, ( actdisyr - curadmyr ) AS years_in_prison
    FROM ildoc_exit
    LIMIT 1000;
    
You can also use special keywords and functions in the SELECT clause.  For example, the keyword "DISTINCT", which only returns any given value in a given column once

- use DISTINCT to only output each inmate's docnbr once, regardless of how many times they have been released from prison:

        SELECT DISTINCT docnbr
        FROM ildoc_exit
        LIMIT 1000;
    
And the "`COUNT()`" aggregation function, passed a value that tells it what you want to count, which returns a count of matching rows rather than a list:

- to just count rows that a query returns, pass count the argument "`*`".  **_Example:_** `COUNT` rows in `ildoc_exit`:

        SELECT COUNT( * )
        FROM ildoc_exit;

- to count `DISTINCT` values, include the `DISTINCT` part of your SELECT clause inside the call to `COUNT()`.  **_Example:_** `COUNT` `DISTINCT` inmate IDs (docnbr) in `ildoc_exit`:

        SELECT COUNT( DISTINCT docnbr )
        FROM ildoc_exit;
    
Also SQL has various "aggregation" functions (`COUNT()` is one) which return a single value for a given input group, for example you can find the minimum (`MIN()`), maximum (`MAX()`), and average (`AVG()`) of numeric values:

    SELECT MIN( curadmyr ) AS earliest_admission_yr, MAX( curadmyr ) AS latest_admission_yr
    FROM ildoc_exit;

## Exercise 1

- Back to the [Table of Contents](#Table-of-Contents)

Use your database client of choice to interact with the database to answer the questions that follow.

For each question, enter:

- The SQL query you used to find the answer.
- The answer to the question.

Questions:

- 1) Find the number of distinct ex-offenders in the `ildoc_exit` database table.
- 2) Find the number of distinct zipcodes in the `ildoc_exit` database table.

Example code:

    SELECT COUNT( DISTINCT( docnbr ) ) AS docnbr_count
    FROM ildoc_exit;

### Exercise 1 work space

#### Question 1 - SQL

#### Question 1 - Answer

#### Question 2 - SQL

#### Question 2 - Answer

## WHERE clauses: Limiting the results

- Back to the [Table of Contents](#Table-of-Contents)

In a SELECT query, you can add a WHERE clause to limit the results (here, limiting to just inmates admitted in 2014):

    SELECT *
    FROM ildoc_exit
    WHERE curadmyr = 2014
    LIMIT 1000;

Where:

- you are making conditional tests, just like in a Python "if" statement.
- EXCEPT here, instead of "==" being the equality operator, it is just "=".
- Comparison operators:

    - "**_`=`_**" - equal to
    - "**_`!=`_**" or "**_`<>`_**" - not equal to
    - "**_`<`_**" - less than
    - "**_`<=`_**" - less-than-or-equal-to
    - "**_`>`_**" - greater than
    - "**_`>=`_**" - greater-than-or-equal-to
    - "**_`LIKE`_**" and "**_`NOT LIKE`_**" - wild-card matching operator, where percent matches 0 or more characters ( "%" ) and an underscore matches any 1 character ( "_" ).
    - "**_`IN( value_list )`_**" and "**_`NOT IN( value_list )`_**" - checks whether the value to the left of the "IN", usually a column's value in a given row, is either IN or NOT IN the list on the right of the IN.
    
An example of using LIKE (include only zip codes that start wth "606" - Chicago):

    SELECT *
    FROM ildoc_exit
    WHERE zipcode LIKE '606%'
    LIMIT 1000;

You can specify multiple conditions for matching in your WHERE clauses, as well, to more precisely filter the results of your query:

    SELECT *
    FROM ildoc_exit
    WHERE curadmyr = 2014 AND admtypr IN ('DFC', 'MVN')
    LIMIT 1000;
    
Note:

- when you are matching a column whose type is numeric, you just put the value in the query, with no quotation marks (just like in Python).
- when you are filtering a string column, you have to include the value you are looking for (the value on the right-hand side of the equal sign) in single-quotes. They must be single-quotes, too.  Unlike in Python, double-quotes have an entirely different meaning than single quotes in SQL, and can cause your query to fail.

Like "None" in Python, the signifier of an unset value in a column for a row is special - NULL.  To check for NULL, you use "IS NULL" or "IS NOT NULL", rather than the "=" or "!=".

    /* find missing values */
    SELECT *
    FROM ildoc_exit
    WHERE exityr IS NULL
    LIMIT 1000;

You can also explicitly cut off the number of results your query returns using the LIMIT keyword.  Just LIMITing to 10 only returns the first 10 results for the query:

    SELECT *
    FROM ildoc_exit
    WHERE curadmyr = 2014 AND admtypr IN ('PVN', 'TPV')
    LIMIT 10;
    
You can also use LIMIT with OFFSET to skip a given number of records. The OFFSET value is the number to skip and LIMIT restricts the number of values returned.

    /* skip 10, the output 15 */
    SELECT *
    FROM ildoc_exit
    WHERE curadmyr = 2014 AND admtypr IN ('PVN', 'TPV')
    OFFSET 10 LIMIT 15;

## Exercise 2

- Back to the [Table of Contents](#Table-of-Contents)

Use your database client of choice to interact with the database to answer the questions that follow.

For each question, enter:

- The SQL query you used to find the answer.
- The answer to the question.

Questions:

- 3) Using any row in the ildoc_exit table that is assigned revised admission type (`admtypr`) "PVN" (parole violation, new sentence), how many years were served?
- 4) Using the rows in the ildoc_exit table that refers to revised admission type (`admtypr`) "PVN" and exit year after 1994, what is the average duration of time spent in prison?

Example code:

    SELECT curadmyr, exityr
    FROM ildoc_exit
    WHERE admtypr = 'PVN'
    LIMIT 1;

### Exercise 2 work space

#### Question 3 - SQL

#### Question 3 - Answer

#### Question 4 - SQL

#### Question 4 - Answer

## JOIN: Connecting multiple tables

- Back to the [Table of Contents](#Table-of-Contents)

We can specify multiple tables in the FROM clause of a select query. This is called a “JOIN”. However, when we do, we need to remember to specify how to match up rows across the two tables. Usually, there is a column that is the same in both tables that can be used to match them up. For much of the course data that will be the hash values of individual's names and SSN.

For examples of joining on SSN hashes, we will now look at the `ildoc_admit` table, which has SSN where `ildoc_exit` does not, and the `il_wage` table.  To start, find wage records that match inmate records based on having the same SSN hash:

    /* Lists earnings of 10 matching records */
    SELECT ia.id AS admit_id, ia.docnbr, iw.id AS wage_id, iw.wage, iw.year, iw.quarter
    FROM ildoc_admit ia
    JOIN il_wage iw
    ON ia.ssn_hash = iw.ssn
    LIMIT 10;

Also, as you can see in the above example, in more complex queries we often give tables temporary short names to make it easy to refer to them.  Temporary short names are added after a given table's name in the FROM clause, separated by a space.  Example: "ia" in "`FROM ildoc_admit ia`".

We can still use regular WHERE clauses in these queries, too, to further filter:

    /* Lists earnings of matching records where individual admitted for violating parole */
    SELECT ia.id AS admit_id, ia.docnbr, ia.admtypo, ia.curadmyr, iw.id AS wage_id, iw.wage, iw.year, iw.quarter
    FROM ildoc_admit ia
    JOIN il_wage iw
    ON ia.ssn_hash = iw.ssn
    WHERE ia.admtypo = 'PVN'
    LIMIT 10;

You can also join more than two tables if you like.

The ability to easily join data from multiple tables together using SQL is one of the most important and useful features of relational databases.  Complex relational data can be broken up into table designs that model the entities and transactions within a system, grouping like information and minimizing repitition, but then SQL allows data from these tables to be combined and flattened to form all kinds of tabular data outputs that are easily used for analysis.

## GROUP BY and aggregate functions

- Back to the [Table of Contents](#Table-of-Contents)

Often, one thing that you want to do is to aggregate over multiple rows. For example, "What are the average earnings for each admission type in 2010?" To do this, use a GROUP BY clause:

    /* Average earnings by revised admission type of matching records for individuals admitted in 2010 */
    SELECT ia.admtypo, AVG( iw.wage ) AS average_earnings
    FROM ildoc_admit ia
    JOIN il_wage iw
    ON ia.ssn_hash = iw.ssn
    WHERE ia.curadmyr = 2010
    GROUP BY ia.admtypo;

_Note: this query might take a minute or two to complete because of the size of the wage table._

There are a number of useful aggregate functions:

- **_SUM( column )_** : Calculate the sum of column for all the rows in each group
- **_AVG( column )_** : Calculate the numeric average for all of the rows in each group
- **_COUNT( column )_** : Count the number of rows in each group
- **_MIN( column ) and MAX( column )_** : Find the minimum or maximum value of column in all the rows in each group

Often, it can be very powerful to combine GROUP BY and table joins, but the queries can quickly become complicated. To simplify the process of building queries that combine GROUP BY and JOIN, first create an SQL query that JOINs correctly to return the individual rows you want to GROUP, then add the aggregate function calls to the SELECT clause and GROUP BY to the end of the SQL statement.

## Exercise 3

- Back to the [Table of Contents](#Table-of-Contents)

Use your database client of choice to interact with the database to answer the questions that follow.

For each question, enter:

- The SQL query you used to find the answer.
- The answer to the question.

Questions:

- 5) Based on information in ildoc_exit, how many individuals were released from IL correctional facilities in 2013? 
- 6) How many individuals were released for each year from 2010-2015?
- 7) How long was the average individual who was released in 2014 in prison for each revised admission type?

Example code:

    SELECT COUNT( * ) AS number_released
    FROM ildoc_exit
    WHERE exityr = 2012;

### Exercise 3 work space

#### Question 5 - SQL

#### Question 5 - Answer

#### Question 6 - SQL

#### Question 6 - answer

#### Question 7 - SQL

#### Question 7 - answer

## ORDER BY

- Back to the [Table of Contents](#Table-of-Contents)

When an SQL query is run, the results are not guaranteed to return in any set order, though basic single-table queries oftern return rows in the order they appear in the database. 

If you want your results ordered a certain way, you use an `ORDER BY` clause to tell the database how to order the rows in the results of a given query.

> Aside: the below queries make use of using an alias for tables and renaming some columns. In the `SELECT` portion of the following query the optional keyword `AS` is used for legibility, whereas the DOC admissions and IL wage record tables are assigned an alias without the `AS` by simply adding a name directly after the table (`ia` and `iw` below, respectively).

A single column example (`ORDER BY` docnbr):

    SELECT ia.id AS admit_id, ia.docnbr, iw.id AS wage_id, iw.wage, iw.year, iw.quarter
    FROM ildoc_admit ia
    JOIN il_wage iw
    ON ia.ssn_hash = iw.ssn
    ORDER BY ia.docnbr
    LIMIT 1000;

In an `ORDER BY` clause one can specify a list of the columns you want to sort the results on, in the order they appear in the list.  The database will first order the rows based on the values in the left-most item in the `ORDER BY` list.  Then as it moves left-to-right through the `ORDER BY` list, when there are duplicates in a given column, if there is another column name in the list to the right of the current column, it will order each set of rows with duplicate values based on the next column named in the `ORDER BY` list.

For example, if you ORDER BY docnbr, then year, then quarter:

    SELECT ia.id AS admit_id, ia.docnbr, iw.id AS wage_id, iw.wage, iw.year, iw.quarter
    FROM ildoc_admit ia
    JOIN il_wage iw
    ON ia.ssn_hash = iw.ssn
    ORDER BY ia.docnbr, iw.year, iw.quarter
    LIMIT 1000;

If there are multiple rows for a given inmate (docnbr), then this query will order the rows for that inmate by year, ascending.  If there are multiple for a given year, the rows for a given year will be ordered by quarter within that year, ascending.

By default, rows are ordered in ASCending order.  After you specify a given column to `ORDER BY`, you can optionally specify either ASC for ascending order, or DESC for descending order.

Using `ORDER BY` with custom column names can be really useful when combined with `GROUP BY`:

    /* Average earnings by admission type of matching records for individuals admitted in 2010 
        order results from highest to lowest earnings */
    SELECT ia.admtypo, AVG( iw.wage ) AS average_earnings
    FROM ildoc_admit ia
    JOIN il_wage iw
    ON ia.ssn_hash = iw.ssn
    WHERE ia.curadmyr = 2010
    GROUP BY ia.admtypo
    ORDER BY average_earnings DESC;

## More questions

- Back to the [Table of Contents](#Table-of-Contents)

In addition to the exercises above, if you'd like to try to answer more of our questions with SQL before you dive into project work, here once again are the questions asked in the pandas introduction notebook:

- The table `ildoc_admit` contains a field `escrisk` which stands for escape risk. The categories are:

    - H: High
    - M: Moderate
    - L: Low
    - P: Pending

    Find the number individuals in `il_admit` with each of these levels of escape risk during the year 2014.  
    
- What is the maximum number of admits a single prisoner has had in the ildoc system?  Let's group the records by the docnbr, then count the number of records in each group.  Sort the values from highest to least, and then get the top value. Which prisoner has the highest number of admits?

- Find the mean, median and minimum value of the number of times each individual prisoner in the ildoc system was admitted to prison.

- Find the distribution of admit counts by year.

<hr />

# PostGIS

- Back to the [Table of Contents](#Table-of-Contents)

The PostGIS extension to PostgreSQL provides a very rich library of spatial functions which can be used directly in the database, and more functionality is added frequently by the community of developers.


## PostGIS examples

- Back to the [Table of Contents](#Table-of-Contents)

Let's combine some of our newfound database knowledge with a few common spatial analysis tasks: (1) how many _things_ are in a given area? (2) how far are _some things_ from _some other things_? and (3) how many _other things_ are within some distance? We'll use the parolees, HUD programs, and Census tract datasets in the examples.

1. Question 1: Which Census Blocks in Chicago had the most parolees in 2015?
2. Question 2: How far are parolees from the nearest HUD program?
3. Question 3: Which 10 parolees' have the most households participating in a HUD program within 5 miles in 2014?

### Example 1: Points in polygons

- Back to the [Table of Contents](#Table-of-Contents)

**Question 1**: Which Census Blocks had the most parolees in 2015?

The two datasets used in the first analysis are
+ Parolees' last know addresses from IL DOC - these data include latitude and longitude coordinates from geocoding with the Census TIGER\Lines data and 85.2% of records were matched at the address level (code snippet below shows geocoding accuracy breakdown)
+ Census Blocks - polygons of census Blocks for 2016, subset to Chicago Metropolitan area

In [ ]:
# SQLAlchemy for database connection
from sqlalchemy import create_engine

# set database connection
conn = create_engine('postgresql://10.10.2.10/appliedda')

# pandas for easy data querying
import pandas as pd

# get number of records by accuracy of geocoded parolee addresses #
qry = "SELECT geocode_level, count(*) recs FROM il_doc_parole GROUP BY geocode_level ORDER BY recs DESC;"
geocode_accuracy = pd.read_sql(qry, conn)

# calculate percentage for each geocode_level
geocode_accuracy['percent'] = geocode_accuracy.recs / geocode_accuracy.recs.sum() *100

# print resulting dataframe
geocode_accuracy

There are a number of distinct tasks we will have the database perform all as one query:
1. Subset the parolee locations to only those in 2015 (in the `WHERE` clause)
2. Join the parolee data with the Block data (in the `FROM` clause)
4. Summarize by block and count the number of parolees (the `GROUP BY` clause coupled with the `count()` aggregator)
5. Show only the top 10 blocks by the count of parolees (using `ORDER BY` and `LIMIT`)

First, we set up the query:

    /* select Block codes and count the number of parolees in each Block */
    SELECT b.geoid10 block_code, count(p.*) parolee_count

    /* set input datasets "b" from the Block data for Chicago and "p" for parolee data */
    FROM tl_2016_16980_tabblock10 b
    JOIN il_doc_parole p
    
    /* join the tables on their geographic relationship, here that parolee locations are within Blocks */
    ON ST_Within(p.geom, b.geom)

    /* filter parolee data using to just 2015 */
    WHERE p.year = 2015

    /* filter out rows where origin and destination are the same zip code. */
    GROUP BY b.geoid10

    /* and let's order by distance so the output is easier to view */
    ORDER BY parolee_count DESC
    
    /* only show the top 10 Blocks by parolee count*/
    LIMIT 10;

Here's the query all together:

    SELECT b.geoid10 block_code, count(p.*) parolee_count
    FROM tl_2016_16980_tabblock10 b JOIN il_doc_parole p
    ON ST_Within(p.geom, b.geom)
    WHERE p.year = 2015
    GROUP BY b.geoid10
    ORDER BY parolee_count DESC
    LIMIT 10;
    
The above query orders by decreasing parolee counts.  If we wanted to see Blocks with the fewest parolees first:

    SELECT b.geoid10 block_code, count(p.*) parolee_count
    FROM tl_2016_16980_tabblock10 b JOIN il_doc_parole p
    ON ST_Within(p.geom, b.geom)
    WHERE p.year = 2015
    GROUP BY b.geoid10
    ORDER BY parolee_count
    LIMIT 10;

And we could always use LIMIT to a different number, or remove the limit all together to return any Block with at least one parolee.

### Example 2: Nearest neighbor distances

- Back to the [Table of Contents](#Table-of-Contents)

In the next question we will again use the Parolee dataset, and additionally we will use the HUD program data. Specifically, we will use the HUD program Block codes (`block_id`) and total households (`hh_tot`) to get a sense of how many households were enrolled in the nearest HUD program for each parolee.

Again there are multiple tasks we will perform all within a single query:
1. Use only parolee and HUD data from 2014 and only parolee where `geocode_level = 'address'`,
2) Subset the datasets to only 
4) Calclate the distances between parolee and HUD locations
5) Keep only distinct records of parolees and the closest HUD location

**Question 2**: How far were parolees from the nearest HUD program in 2014?

    /* select the unique records of parolees, their Unique identifier, number of HH in this Block, and the distance between the parolees' locations and the nearest Block centroid that has HUD participating households */
    SELECT DISTINCT ON (p.id) p.person_id, h.hh_tot, ST_Distance(p.geom, h.geom) AS distance

    /*** define the input ***/
    /* parolee data and HUD data */
    FROM il_doc_parole AS p, hud_household_block_data AS h
    
    /* consider only the 2014 records for each dataset, only parolee locations that successfully geocoded, and HUD household data that has Block level locations */
    WHERE year = 2014 AND geocode_level = 'address' AND h.admin_yr = 2014 AND h.geom IS NOT NULL


    /* ORDER BY p.id so that DISTINCT works and second by distance to only retain the closest HUD records; LIMIT to just 10 so thousands of records are not printed out (~25k parolee addresses in 2014): */
    ORDER BY p.id, distance 
    LIMIT 10;

So, here's a compacted version of the query that we could copy and run (BUT DON'T, it's slow):

    SELECT DISTINCT ON (p.id) p.person_id, h.hh_tot, ST_Distance(p.geom, h.geom) AS distance
    FROM il_doc_parole AS p, hud_household_block_data AS h
    WHERE year = 2014 AND geocode_level = 'address' AND h.admin_yr = 2014 AND h.geom IS NOT NULL
    ORDER BY p.id, distance 
    LIMIT 10;
    
The above query takes ~5-10 minutes to complete. Why? Because it needs to calculate the distance between all the ~24k parolee locations and ~6.5k HUD Blocks (after the filters in the `WHERE` clause).

Using psql the above query looks like this:

    appliedda=> SELECT DISTINCT ON (p.id) p.person_id, h.hh_tot, ST_Distance(p.geom, h.geom) AS distance
    appliedda->     FROM il_doc_parole AS p, hud_household_block_data AS h
    appliedda->     WHERE year = 2014 AND geocode_level = 'address' AND h.admin_yr = 2014 AND h.geom IS NOT NULL
    appliedda->     ORDER BY p.id, distance 
    appliedda->     LIMIT 10;
     
     -- and output will look something like the below
     person_id | hh_tot |       distance       
    -----------+--------+----------------------
             xx |    xx      xx
             xxxxxxxxxxxxxxxxxxxx
             xxxxxxxxxxxxxxxxxxxx
             xxxxxxxxxxxxxxxxxxxx
             xxxxxxxxxxxxxxxxxxxx
    (10 rows)

Looking at the above distances the values are...  difficult to interpret. This is because of geographic projections (aka Coordinate Reference Systems or CRS, see the [addendum](#Addendum---Some-other-useful-SQL-+-Python-snippets)). The geometry column we used ('geom') is in projection "NAD83" so the resulting distances are in "degrees". Let's redo it with the US Atlas projection which is in meters, column "geom_2163" for parolee data and "geom_pnt_2163" for HUD data (this is actually the centroid of the associated Block).

Note: please see the [Add spatial info](../../data/Add-spatial-info-to-appliedda-tables.ipynb) notebook in the `data` folder for details of how geometries were added to these tables.

So, let's redo the query but make the distances more useful. Additionally we are going to `JOIN` the tables based on a distance parameter so the database does not need to calculate all ~24k * ~6k distances - check out the difference in the `FROM` clause below with the previous query above. The `ST_DWithin()` function returns `TRUE` when the first geometry is within the specified distance of the second geometry, below it is set to two miles (since our projection is in meters, we multiply 1609.344 by the number of miles we want).

    SELECT DISTINCT ON (p.id) p.person_id, h.hh_tot, ST_Distance(p.geom_2163, h.geom_pnt_2163)/1609.344 AS distance_miles
    FROM il_doc_parole AS p
    JOIN hud_household_block_data AS h
    ON ST_DWithin(p.geom_2163, h.geom_pnt_2163, 5*1609.344)
    WHERE year = 2014 AND geocode_level = 'address' 
        AND h.admin_yr = 2014 AND h.geom_pnt_2163 IS NOT NULL
    ORDER BY p.id, distance_miles
    LIMIT 10;

Now the distances are much more readily interpretable as miles.

<hr />

# Addendum - Other useful SQL

- Back to the [Table of Contents](#Table-of-Contents)

Below are a handful of examples which may be useful:

1. [Listing tables and columns](#Listing-tables-and-columns)
2. [Modifying a database](#Modifying-the-database)
2. [Coordinate Reference Systems](#Coordinate-Reference-Systems)
3. [Get spatial information from PostGIS](#Get-spatial-information-from-PostGIS)

## Listing tables and columns

- Back to the [Table of Contents](#Table-of-Contents)
- Back to the [Addendum](#Addendum---Other-useful-SQL)

One thing that can be useful is seeing what tables (and views etc) exist in a database. PostgreSQL has a few different places to get information about what exists in the database, but in the following example we will make use of the `pg_class` table.

Task: list all tables that start with "il":

    SELECT relname, relkind
    FROM pg_class
    WHERE relname LIKE 'il%';
    
Limit to only tables (relkind = 'r'):

    SELECT relname, relkind
    FROM pg_class
    WHERE relname LIKE 'il%'
    AND relkind = 'r';

## Modifying the database

- Back to the [Table of Contents](#Table-of-Contents)
- Back to the [Addendum](#Addendum---Other-useful-SQL)

In addition to retrieving information from an existing database, you can also insert data into a database, update existing rows, and delete records using SQL. Permissions on databases generally will not allow you to modify the databases, but below are example queries of how you could modify a database to which you have 'WRITE' permission.

Here are some example queries:

- **CREATE**: Adding a table to a database

        CREATE TABLE data (
        ID int(11) serial primary key, 
        name_first varchar(20)
        name_last varchar(30))

- **INSERT**: Adding a row to a table

        INSERT INTO data
        (name_first, name_last)
        VALUES ('Christina', 'Jones')

- **UPDATE**: Changing data that is already in a table

        UPDATE data
        SET name_last = 'Johnson'
        WHERE name_first = 'Christina'
        
- **ALTER TABLE**: Changing the structure of an existing table

        ALTER TABLE data
            ADD COLUMN gender VARCHAR(1) DEFAULT 'F'

- **DELETE**: Removing one or more rows from a table

        DELETE FROM cjones.data
        WHERE name_last = 'Johnson'

- **DELETE**: removing table from database

        DROP TABLE cjones.data


Lastly, you can also CREATE a table using an existing table. 

- **CREATE**: Adding a table to a database (NOTE: creating a table this way is one of the few times **`as`** is required by PostgreSQL)

        CREATE TABLE parole_violator_exit_2012 AS (
        SELECT * FROM ildoc_exit
        WHERE exityr = 2012
        and admtypr IN ('PVN', 'TPV'));

## Coordinate Reference Systems

- Back to the [Table of Contents](#Table-of-Contents)
- Back to the [Addendum](#Addendum---Other-useful-SQL)

This is an extremely brief overview of Coordinate Reference Systems ("CRS", also known as "projections"). Because Earth is a spheroid (like a sphere, but problematically bumby) people have created a lot of ways to deal with representing Earth's three dimensional surface in flat, two-dimensional space. Projections are basically math that (1) describes how information in a given dataset relates to the rest of the world and (2) usually creates a 'flat' surface on which data can be analyzed using more common algorithms (eg Euclidean geometry). 

A projection has two main components:

1. The "datum" provides the context of where in the world the projection applies. The two most common in the US are WGS84 (World Geodesic Survey of 1984 - if you know your home or city latitude and longitude it's probably in WGS84) and NAD83 (North American Datum 1983), and
2. The transformation algorithm - this has many parts that we don't need to cover here. If you're interested in learning more here's a couple resources: [NOAA tutorial](https://coast.noaa.gov/digitalcoast/training/datums.html) and [ArcGIS description](http://webhelp.esri.com/arcgisdesktop/9.3/index.cfm?TopicName=About%20map%20projections)

Here's some examples of global projections (thanks to [xkcd](https://xkcd.com/977/)):
![map-projections](./images/map_projections_xkcd.png)

There are many different continental, regional, and country specific projections created by different agencies. The international spatial data and analysis community has created standard set of codes to easily tell different software programs what reference system your data are in or to which projection you are transforming your data.
A couple good resource for finding coordinate reference systems are [epsg.io](http://epsg.io/) and [spatialreference.org](http://spatialreference.org/).

If you get spatial data from somewhere, the metadata **_should_** include what Reference System the data are in. If your data are delivered as a `shapefile` (which is actually a collection of at least 4 individual files) then the `.prj` file includes the projection definition in "well known text" (WKT) format, in PostGIS this looks something like:

    GEOGCS["NAD83",
        DATUM["North_American_Datum_1983",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]
            ],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6269"]
        ],
        PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]
    ],
    UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4269"]]

Another common way programs understand projections is in what's called the "proj4" format, in PostGIS it looks like this:

    +proj=longlat +datum=NAD83 +no_defs
    
PostGIS handles projections via some math and programming we don't need to worry about here and the `spatial_ref_sys` table (ie "spatial reference systems"), which contains both the WKT and proj4 definitions as well as a unique identifier: `srid`. Generally the `spatial_ref_sys` table is populated with many (~3-4 thousand) projections, however sometimes data will come in a format unknown to the database. If that happens you can usually find an `INSERT` statement on epsg.io to add the projection to your database. Here's what that looks like for the "NAD83 / New York Long Island (ft)" coordinate system:

    INSERT into spatial_ref_sys (srid, auth_name, auth_srid, proj4text, srtext) 
        values ( 2263, 'EPSG', 2263, 
            '+proj=lcc +lat_1=41.03333333333333 +lat_2=40.66666666666666 +lat_0=40.16666666666666 +lon_0=-74 
                +x_0=300000.0000000001 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=us-ft +no_defs ', 
            'PROJCS["NAD83 / New York Long Island (ftUS)",GEOGCS["NAD83",
                DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
                TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,
                AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],
                AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],
                PARAMETER["standard_parallel_1",41.03333333333333],
                PARAMETER["standard_parallel_2",40.66666666666666],
                PARAMETER["latitude_of_origin",40.16666666666666],PARAMETER["central_meridian",-74],
                PARAMETER["false_easting",984250.0000000002],PARAMETER["false_northing",0],
                UNIT["US survey foot",0.3048006096012192,AUTHORITY["EPSG","9003"]],
                AXIS["X",EAST],AXIS["Y",NORTH],
                AUTHORITY["EPSG","2263"]]'
        );



## Get spatial information from PostGIS

- Back to the [Table of Contents](#Table-of-Contents)
- Back to the [Addendum](#Addendum---Other-useful-SQL)

In PostGIS the `geometry_columns` view maintains information about what geometry data exists in the tables in the database, so there's a simple way to see _all_ the tables that have spatial data like so:

    SELECT f_table_name
    FROM geometry_columns
    GROUP BY f_table_name
    ORDER BY f_table_name;
    
And there is also a way to see what spatial columns exist in a given table, along with their SRID and datatype:

    SELECT f_geometry_column, srid, type
    FROM geometry_columns WHERE
    f_table_name = 'tl_2016_us_zcta510';

> Reminder: the SRID is the Spatial Reference Identifier, a unique code used in the `spatial_ref_sys` table as described in the Coordinate Reference System section above